In [5]:
import pandas as pd 
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import faiss


In [16]:
df = pd.read_csv("movies_list.csv")
df['genre'].value_counts()
pd.set_option('display.max_rows', None)

In [17]:
df['genre'].value_counts()

genre
драма                                 5773
комедия                               1796
боевик                                1684
документальный                        1603
мультфильмы                           1502
биографический                         803
детектив                               777
триллер                                441
война                                  378
мелодрама                              206
ужасы                                  190
приключения                            131
вестерн                                115
семейный                                71
музыка                                  69
короткометражный                        43
фантастика                              32
исторический                            23
мюзикл                                  18
мистика                                 17
фэнтези                                 13
биографический, драма                    6
для взрослых                             5
военн

In [18]:
# Пример словаря для замены редких/устаревших жанров
genre_mapping = {
    "нуар": "драма",
    "вестерн": "боевик",
    "мюзикл": "комедия",
    "биография": "драма",
    "исторический": "драма",
    "эротика": "драма",
    "артхаус": "драма",
    "концерт": "музыка",
    'музыкальный': 'музыка',
    "ток-шоу": "документальный",
    "экспериментальный": "драма",
    "спортивный": "драма",
    "семейный": "приключения",
    "военный": "война",
    'реалити-шоу': 'документальный',
    "игровое шоу": 'документальный',
    "фильм-катастрофа": "боевик",
    'аниме': 'мультфильмы',
    'криминал': 'драма'
}

# Разделяем жанры
df['genre_list'] = df['genre'].fillna('').apply(lambda x: [g.strip().lower() for g in x.split(',') if g.strip()])

# Заменяем по словарю
def map_genres(genres):
    return [genre_mapping.get(g, g) for g in genres]

df['genre_list'] = df['genre_list'].apply(map_genres)

In [19]:
df['genre'] = df['genre_list'].apply(lambda lst: ', '.join(sorted(set(lst))))

In [20]:
df['genre'].value_counts()

genre
драма                       5802
комедия                     1815
боевик                      1799
документальный              1606
мультфильмы                 1502
биографический               803
детектив                     777
триллер                      441
война                        382
мелодрама                    206
приключения                  202
ужасы                        190
музыка                        70
короткометражный              43
фантастика                    32
мистика                       17
фэнтези                       13
биографический, драма          7
война, драма                   7
для взрослых                   5
боевик, драма                  4
мультфильмы, приключения       4
драма, приключения             3
приключения, фэнтези           2
детектив, триллер              2
триллер, ужасы                 2
комедия, мелодрама             2
драма, триллер                 2
мультфильмы, фэнтези           2
комедия, мультфильмы           2
спор

In [21]:
df.to_csv('movies_list.csv', index=False)

In [3]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2") 

/home/polina/miniforge3/envs/fixed_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
embeddings = model.encode(df['description'].tolist(),show_progress_bar=True)

Batches: 100%|██████████| 493/493 [00:27<00:00, 18.17it/s]


In [5]:
embeddings = np.array(embeddings).astype("float32")

In [6]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [4]:
import pandas as pd
import numpy as np
import faiss
import time
from sentence_transformers import SentenceTransformer
import wikipediaapi
from tqdm import tqdm

# --- Настройки Wikipedia ---
USER_AGENT = "MovieRecommendationBot/1.0 (https://example.com/contact)"
wiki = wikipediaapi.Wikipedia(language="ru", user_agent=USER_AGENT)

# --- Получение сюжета из Википедии ---
def get_plot(title):
    try:
        # Добавляем задержку, чтобы избежать блокировки
        time.sleep(0.5)
        
        page = wiki.page(title)
        
        # Проверяем существование страницы с дополнительной защитой
        if not hasattr(page, 'pageid') or getattr(page, 'pageid', -1) == -1:
            return ""
            
        # Ищем разделы с сюжетом (пробуем разные варианты названий)
        section_titles = ["Сюжет", "Содержание", "Фабула", "Plot"]
        for section_title in section_titles:
            if section_title in page.sections:
                return page.sections[section_title].text
        
        # Если раздел не найден, возвращаем пустую строку
        return ""
    
    except Exception as e:
        print(f"Ошибка при получении сюжета для '{title}': {str(e)}")
        return ""

# --- Шаг 1: Загрузка данных и добавление сюжетов ---
df = pd.read_csv("movies_list.csv")
df["description"] = df["description"].fillna("")

tqdm.pandas(desc="📖 Получение сюжетов с Википедии")
df["plot"] = df["movie_title"].progress_apply(get_plot)

# --- Шаг 2: Объединённый текст для эмбеддингов ---
df["full_text"] = df["description"] + "\n" + df["plot"]

# --- Шаг 3: Генерация эмбеддингов ---
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
texts = df["full_text"].tolist()
embeddings = model.encode(texts, show_progress_bar=True)
embeddings = np.array(embeddings).astype("float32")

# --- Шаг 4: Нормализация эмбеддингов (для косинусного сходства) ---
embeddings /= np.linalg.norm(embeddings, axis=1, keepdims=True)

# --- Шаг 5: Создание FAISS индекса ---
index = faiss.IndexFlatIP(embeddings.shape[1])  # IP == cosine sim (после нормализации)
index.add(embeddings)

# --- Шаг 6: Сохранение ---
np.save("movie_vectors1.npy", embeddings)
faiss.write_index(index, "index1.bin")
df.to_csv("movies_with_plots.csv", index=False)

print("✅ Готово! Всё сохранено.")



📖 Получение сюжетов с Википедии:   6%|▋         | 1016/15756 [15:58<3:51:42,  1.06it/s]


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# Загрузка данных
df = pd.read_csv("movies_with_plots.csv")
embeddings = np.load("movie_vectors1.npy")
index = faiss.read_index("index1.bin")

# Загрузка той же модели
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


In [ ]:
def recommend_by_text(user_text, top_k=5):
    query_vec = model.encode([user_text])
    query_vec = query_vec / np.linalg.norm(query_vec, axis=1, keepdims=True)
    scores, indices = index.search(query_vec.astype("float32"), top_k)

    print(f"🎯 Рекомендации по описанию: '{user_text}'\n")
    for i in indices[0]:
        print(f"→ {df.iloc[i]['movie_title']} ({df.iloc[i].get('year', 'год не указан')})")
        print(f"Описание: {df.iloc[i]['description'][:200]}...")
        print("-" * 50)



In [ ]:
recommend_by_text("киберпанк, мрачное будущее, виртуальная реальность", top_k=5)


In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
GROQ_API_KEY="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"
# Для Groq Cloud
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# === Настройки модели ===
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
GROQ_MODEL = "mixtral-8x7b-32768"  # или "llama3-70b-8192"

# === Загрузка данных ===
@st.cache_data
def load_data():
    df = pd.read_csv("movies_list.csv")
    return df

# === Загрузка модели и индекса ===
@st.cache_resource
def load_model_and_index():
    model = SentenceTransformer(MODEL_NAME)
    vectors = np.load("movie_vectors.npy")
    index = faiss.read_index("index.bin")
    return model, index, vectors

# === Функция поиска фильмов ===
def find_similar_movies(query, model, index, df, top_k=5):
    query_vec = model.encode([query]).astype('float32')
    D, I = index.search(query_vec, top_k)
    return df.iloc[I[0]]

# === Подключение к Groq Cloud ===
def get_groq_llm():
    return ChatGroq(
        model=GROQ_MODEL,
        temperature=0.7,
        max_tokens=1000,
        timeout=None,
        api_key=os.getenv("GROQ_API_KEY")
    )

# === Форматирование результатов для LLM ===
def format_docs(docs):
    formatted = []
    for i, row in docs.iterrows():
        info = f"""
{i+1}. **{row['movie_title']}** ({row.get('year', '?')})
   Жанр: {row.get('genre', 'Не указан')}
   Описание: {row.get('description', '')[:200]}...
"""
        formatted.append(info)
    return "\n".join(formatted)

# === RAG цепочка с Groq Cloud ===
def create_rag_chain(model, index, df):
    llm = get_groq_llm()

    prompt = ChatPromptTemplate.from_messages([
        ("system", """Ты кинокритик с чувством юмора.
Твоя задача: 
- Проанализировать фильмы из контекста
- Дать шутливые, но точные рекомендации
- Объяснить, почему они подходят под запрос
- Можно добавить мемы или сравнения с известными фильмами

Если фильмы не найдены — тоже скажи об этом, но с юмором 😊"""),
        ("human", """
🔍 Запрос пользователя: "{question}"
🎬 Вот фильмы, которые я нашёл:

{context}

💬 Ответ:""")
    ])

    def retrieve_and_format(query):
        results = find_similar_movies(query, model, index, df, top_k=5)
        if len(results) == 0:
            return {"context": "Ничего не нашлось...", "question": query}
        return {"context": format_docs(results), "question": query}

    rag_chain = (
        RunnablePassthrough(input=lambda x: x["query"])
        | retrieve_and_format
        | prompt
        | llm
        | StrOutputParser()
    )
    return rag_chain

# === Streamlit UI ===
import os
os.environ["GROQ_API_KEY"] = st.secrets.get("GROQ_API_KEY", "ваш_ключ")

st.set_page_config(page_title="🎬 Умные рекомендации", layout="wide")
st.title("🤖 Умный поиск фильмов через Groq Cloud")

df = load_data()
model, full_index, vectors = load_model_and_index()

rag_chain = create_rag_chain(model, full_index, df)

# === Ввод пользователя ===
user_query = st.text_input("Введите запрос, например: 'Фильм про любовь в стиле аниме'")
if st.button("🔍 Найти и спросить ИИ"):
    if not user_query.strip():
        st.warning("⚠️ Пожалуйста, введите запрос!")
    else:
        with st.spinner("🧠 Думаю над этим..."):
            try:
                answer = rag_chain.invoke({"query": user_query})
                st.markdown("### 💬 Ответ от ИИ:")
                st.markdown(answer)
            except Exception as e:
                st.error(f"❌ Ошибка: {e}")

In [8]:
# Загрузка данных
df = pd.read_csv("movies_list.csv")

# Предобработка описаний
df["description"] = df["description"].fillna("")

# Загрузка модели
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Генерация эмбеддингов
embeddings = model.encode(df["description"].tolist(), show_progress_bar=True)
embeddings = np.array(embeddings).astype("float32")

# Нормализация для косинусного сходства
embeddings /= np.linalg.norm(embeddings, axis=1, keepdims=True)

# Создание индекса на основе скалярного произведения (эквивалент косинусному сходству при нормализации)
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product == Cosine similarity при нормализации
index.add(embeddings)

# Сохранение
np.save("movie_vectors.npy", embeddings)
faiss.write_index(index, "index.bin")

/home/polina/miniforge3/envs/fixed_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|██████████| 493/493 [00:25<00:00, 19.21it/s]


In [18]:
# Запрос пользователя
user_query = "Новогодняя ночь"
query_vector = model.encode([user_query],convert_to_numpy=True).astype("float32")

# Векторизуем запрос
#query_embedding = model.encode([user_query])

# Считаем схожесть запроса с каждым сериалом
#similarities = cosine_similarity(query_embedding, embeddings)[0]  # получаем 1D-массив

# Получим индексы топ-5 самых похожих сериалов
#top_n = 5
#top_indices = similarities.argsort()[-top_n:][::-1]  # сортировка по убыванию

# Выводим названия и описания похожих сериалов
# for i in top_indices:
#     print(f"\n🎬 Название: {df.iloc[i]['tvshow_title']}")
#     print(f"📄 Описание: {df.iloc[i]['description']}")
#     print(f"🔍 Схожесть: {similarities[i]:.3f}")

k = 9
distances, indices = index.search(query_vector, k)

# Выводим результаты
for i in indices[0]:
    print(f"\n🎬 Название: {df.iloc[i]['movie_title']}")
    print(f"📄 Описание: {df.iloc[i]['description']}")


🎬 Название: С Новым годом!
📄 Описание: Друзья собираются, чтобы отпраздновать Новый год. В ходе вечера они решают сыграть в игру и положить свои телефоны на общее обозрение. Во время игры раскроется много секретов, разобьются сердца и сломаются судьбы. С Новым годом!

🎬 Название: О чем еще говорят мужчины
📄 Описание: Так случилось, что почти весь день 31 декабря наши герои проводят вместе, Как они проводят время накануне Нового года? Разумеется, в разговорах. О чем? Во-первых, конечно, о женщинах – тема-то неисчерпаемая. А еще о…

🎬 Название: Обратная связь
📄 Описание: Канун Нового года. Старая компания снова в сборе и собирается весело отметить праздник. Но атмосфера в доме вновь накаляется до предела.

🎬 Название: Декалог, три
📄 Описание: Канун Рождества - ночь, когда вся семья вместе, и никто не хочет оставаться один. Эва соблазняет Януша, ее бывшего любовника, под любым предлогом выйти из дома и пробыть с ней всю ночь. Януш хочет вернуться домой, но Эва определенно решительна… 

🎬